In [1]:
import pandas as pd
import numpy as np
import warnings
import ray

In [2]:
ray.init()
def Añadir_Paths(start,end):
    basepath = "/media/jonathan/JHON/Tesis/DatosTesis/DatosDesplazamiento/maestra1-mitma-distritos/ficheros-diarios/%.4d%.2d%.2d_maestra_1_mitma_distrito.csv"
    dates = pd.date_range(start, end)
    paths = [basepath % (timestamp.year, timestamp.month, timestamp.day) for timestamp in dates]
    return paths

@ray.remote
def trayecto_values(path):
    data = pd.read_csv(path, dtype={"fecha": str,
                                    "origen": str,
                                    "destino": str}, sep='|')
    
    data = data[['fecha','origen','destino','periodo','distancia','viajes']] 
#     print(data.head())
    data = data[data.distancia >= "002-005"]    
    
    df_trayecto = data.loc[data['origen'] == '2807905'] 
    df_trayecto = df_trayecto.loc[df_trayecto['destino'] == '2807901']     
    
    return df_trayecto

2023-10-03 18:50:15,268	INFO services.py:1272 -- View the Ray dashboard at http://127.0.0.1:8265


In [3]:
start = "20200621"
end = "20200731"

trayecto_dfs = []
df_trayecto = pd.DataFrame({})

Rutas = Añadir_Paths(start,end)
for path in Rutas:
    trayecto_dfs.append(trayecto_values.remote(path))

trayecto_dfs = ray.get(trayecto_dfs)
for df in trayecto_dfs:
#    contador += 1
    df_trayecto = df_trayecto.append(df)

/tmp/ipykernel_10326/754034029.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_trayecto = df_trayecto.append(df)
/tmp/ipykernel_10326/754034029.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_trayecto = df_trayecto.append(df)
/tmp/ipykernel_10326/754034029.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_trayecto = df_trayecto.append(df)
/tmp/ipykernel_10326/754034029.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_trayecto = df_trayecto.append(df)
/tmp/ipykernel_10326/754034029.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conca

In [4]:
df_trayecto = df_trayecto[['fecha','origen','destino','periodo','distancia','viajes']]
df_trayecto.head()

,fecha,origen,destino,periodo,distancia,viajes
2702468,20200621,2807905,2807901,0,002-005,16.335
2702469,20200621,2807905,2807901,1,002-005,16.335
2702470,20200621,2807905,2807901,1,005-010,5.445
2702471,20200621,2807905,2807901,2,002-005,16.335
2702472,20200621,2807905,2807901,3,002-005,16.335


In [5]:
# df_trayecto.to_csv("trayectoFeb_Jul.csv")